In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 77.8 MB/s eta 0:00:00


## GDC

Grouping the Amino Acids
Below are some common groupings based on physicochemical properties:

Hydrophobic (H): A, F, G, I, L, M, P, V, W
Hydrophilic (P): D, E, K, N, Q, R
Neutral (N): C, H, S, T, Y

In [ ]:
import pandas as pd
from Bio import SeqIO

# Define amino acid groups (e.g., based on hydrophobicity or charge)
amino_acid_groups = {
    'H': ['A', 'F', 'G', 'I', 'L', 'M', 'P', 'V', 'W'],  # Hydrophobic
    'P': ['D', 'E', 'K', 'N', 'Q', 'R'],                # Hydrophilic
    'N': ['C', 'H', 'S', 'T', 'Y']                      # Neutral
}

# Create all possible group dipeptides from the groups defined above
groups = list(amino_acid_groups.keys())
group_dipeptides = [g1 + g2 for g1 in groups for g2 in groups]

# Function to map an amino acid to its group
def map_to_group(aa):
    for group, aa_list in amino_acid_groups.items():
        if aa in aa_list:
            return group
    return None

# Function to compute GDC features
def compute_gdc_features(sequence):
    sequence = sequence.upper()
    seq_len = len(sequence)

    # Initialize the GDC vector for each group dipeptide
    gdc_vector = {dipeptide: 0 for dipeptide in group_dipeptides}

    # Convert each amino acid in the sequence to its group and calculate dipeptide composition
    for i in range(seq_len - 1):
        aa1, aa2 = sequence[i], sequence[i+1]
        group1, group2 = map_to_group(aa1), map_to_group(aa2)

        if group1 and group2:
            dipeptide = group1 + group2
            gdc_vector[dipeptide] += 1

    # Normalize the counts by the total number of dipeptides in the sequence
    total_dipeptides = seq_len - 1
    for dipeptide in gdc_vector:
        gdc_vector[dipeptide] /= total_dipeptides

    return list(gdc_vector.values())

# Function to process a FASTA file and extract GDC features, saving them to a CSV
def process_gdc(fasta_file, output_csv):
    sequences = []

    # Read the sequences from the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(record.seq))

    # Compute GDC features for each sequence
    gdc_features = [compute_gdc_features(seq) for seq in sequences]

    # Convert GDC features to DataFrame
    gdc_df = pd.DataFrame(gdc_features, columns=group_dipeptides)

    # Save to CSV
    gdc_df.to_csv(output_csv, index=False)
    print(f"GDC features saved to {output_csv}")

# Input FASTA file paths
main_p = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/POSITIVE_main (1).fasta"
main_n = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/NEGATIVE_main (1).fasta"
validation_p = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/POSITIVE_validation (1).fasta"
validation_n = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/NEGATIVE_validation (1).fasta"

# Output CSV file paths
output_main_p = "/content/gdc_main_positive.csv"
output_main_n = "/content/gdc_main_negative.csv"
output_validation_p = "/content/gdc_validation_positive.csv"
output_validation_n = "/content/gdc_validation_negative.csv"

# Process and save GDC features for each dataset
process_gdc(main_p, output_main_p)
process_gdc(main_n, output_main_n)
process_gdc(validation_p, output_validation_p)
process_gdc(validation_n, output_validation_n)


GDC features saved to /content/gdc_main_positive.csv
GDC features saved to /content/gdc_main_negative.csv
GDC features saved to /content/gdc_validation_positive.csv
GDC features saved to /content/gdc_validation_negative.csv


In [ ]:
import pandas as pd

# Define function to check columns, length, and null values in the dataset
def check_dataset_info(file_path):
    try:
        # Load the CSV file
        df = pd.read_csv(file_path)

        # Get the number of columns and length (number of rows)
        num_columns = len(df.columns)
        num_rows = len(df)

        # Check for null values
        null_values = df.isnull().sum().sum()  # Count total number of null values

        # Return the information
        return num_columns, num_rows, null_values
    except FileNotFoundError:
        return "File not found: " + file_path

# Paths for input CSV files (update these paths if necessary)
main_p = "/content/gdc_main_positive.csv"
main_n = "/content/gdc_main_negative.csv"
validation_p = "/content/gdc_validation_positive.csv"
validation_n = "/content/gdc_validation_negative.csv"

# Check columns, length, and null values for each file
main_p_info = check_dataset_info(main_p)
main_n_info = check_dataset_info(main_n)
validation_p_info = check_dataset_info(validation_p)
validation_n_info = check_dataset_info(validation_n)

# Print the results
print(f"Main Positive Dataset - Columns: {main_p_info[0]}, Rows: {main_p_info[1]}, Null Values: {main_p_info[2]}")
print(f"Main Negative Dataset - Columns: {main_n_info[0]}, Rows: {main_n_info[1]}, Null Values: {main_n_info[2]}")
print(f"Validation Positive Dataset - Columns: {validation_p_info[0]}, Rows: {validation_p_info[1]}, Null Values: {validation_p_info[2]}")
print(f"Validation Negative Dataset - Columns: {validation_n_info[0]}, Rows: {validation_n_info[1]}, Null Values: {validation_n_info[2]}")


Main Positive Dataset - Columns: 9, Rows: 582, Null Values: 0
Main Negative Dataset - Columns: 9, Rows: 582, Null Values: 0
Validation Positive Dataset - Columns: 9, Rows: 150, Null Values: 0
Validation Negative Dataset - Columns: 9, Rows: 150, Null Values: 0


# **Deep learning approach combining Conv1D, LSTM, and Dense layers**

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout, BatchNormalization

# Load AAC feature datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Create labels
main_p['label'] = 1
main_n['label'] = 0
validation_p['label'] = 1
validation_n['label'] = 0

# Combine datasets
train_data = pd.concat([main_p, main_n], ignore_index=True)
val_data = pd.concat([validation_p, validation_n], ignore_index=True)

# Separate features and labels
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values
X_val = val_data.drop(columns=['label']).values
y_val = val_data['label'].values

# Reshape input for Conv1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

# Model Architecture
model = Sequential()

# 1. Stacked Conv1D layers with BatchNormalization and Dropout
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# Adjusted Conv1D to handle small input size with padding
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
# Removed MaxPooling1D here to prevent reduction of the dimension further
model.add(Dropout(0.3))

model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
# Removed MaxPooling1D here to prevent further reduction of the dimension
model.add(Dropout(0.3))

# 2. LSTM layer for sequential dependencies
model.add(LSTM(64, return_sequences=False, activation='relu'))

# 3. Dense Layers for final prediction with Dropout for regularization
model.add(Dense(128, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 9, 64)               │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 9, 64)               │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 4, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 4, 128)              │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 4, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 4, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 4, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 224,129 (875.50 KB)

 Trainable params: 223,233 (872.00 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 22s 232ms/step - accuracy: 0.5657 - loss: 0.6525 - val_accuracy: 0.5000 - val_loss: 0.6920
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6734 - loss: 0.5804 - val_accuracy: 0.5000 - val_loss: 0.6881
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7306 - loss: 0.5320 - val_accuracy: 0.5400 - val_loss: 0.6849
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6986 - loss: 0.5453 - val_accuracy: 0.5700 - val_loss: 0.6712
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7113 - loss: 0.5161 - val_accuracy: 0.5000 - val_loss: 0.9036
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7502 - loss: 0.4960 - val_accuracy: 0.5767 - val_loss: 0.6808
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7362 - loss: 0.4905 - val_accuracy: 0.5400 - val_loss: 1.2598
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7694 - loss: 0.4742 - val_accuracy: 0.5600 

In [ ]:
# Print final training and validation accuracy
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")

Final Training Accuracy: 0.9227
Final Validation Accuracy: 0.8367


# **Hybrid CNN-LSTM Model for Cell-Penetrating Peptide Classification**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Label the datasets
main_p['label'] = 1
main_n['label'] = 0
validation_p['label'] = 1
validation_n['label'] = 0

# Combine datasets
train_data = pd.concat([main_p, main_n], ignore_index=True)
val_data = pd.concat([validation_p, validation_n], ignore_index=True)

# Separate features and labels
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values
X_val = val_data.drop(columns=['label']).values
y_val = val_data['label'].values

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Reshape data for Conv1D input
X_train = X_train[..., np.newaxis]  # Adding channel dimension
X_val = X_val[..., np.newaxis]      # Adding channel dimension

# Model Architecture
model = Sequential()

# Conv1D layers with reduced kernel sizes and fewer pooling layers
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv1D(filters=256, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# LSTM layer for sequential dependencies
model.add(LSTM(64, return_sequences=False, activation='relu'))

# Dense Layers for final prediction with Dropout for regularization
model.add(Dense(128, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, verbose=1)

# Evaluate the model on the validation data
val_predictions = (model.predict(X_val) > 0.5).astype(int)
accuracy = accuracy_score(y_val, val_predictions)

print("\nValidation Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_val, val_predictions))

# Save the trained model
model.save("cell_penetrating_peptide_model.h5")
print("\nModel saved as 'cell_penetrating_peptide_model.h5'")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)                    │ (None, 8, 64)               │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 8, 64)               │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 4, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 4, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 3, 128)              │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 3, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 2, 256)              │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 2, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 183,105 (715.25 KB)

 Trainable params: 182,209 (711.75 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 17s 201ms/step - accuracy: 0.5835 - loss: 0.6732 - val_accuracy: 0.6433 - val_loss: 0.6870
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6771 - loss: 0.5803 - val_accuracy: 0.5967 - val_loss: 0.6774
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6913 - loss: 0.5775 - val_accuracy: 0.5800 - val_loss: 0.6640
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7250 - loss: 0.5328 - val_accuracy: 0.6033 - val_loss: 0.6357
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7325 - loss: 0.5069 - val_accuracy: 0.6333 - val_loss: 0.6068
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7429 - loss: 0.5149 - val_accuracy: 0.7300 - val_loss: 0.5516
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7558 - loss: 0.4775 - val_accuracy: 0.7700 - val_loss: 0.5106
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7723 - loss: 0.4530 - val_accuracy: 0.7767


Validation Accuracy: 0.8433333333333334

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.87      0.85       150
           1       0.87      0.81      0.84       150

    accuracy                           0.84       300
   macro avg       0.84      0.84      0.84       300
weighted avg       0.84      0.84      0.84       300


Model saved as 'cell_penetrating_peptide_model.h5'


# ALL Algorithm for this Dataset

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam



/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")


In [ ]:
# Label the datasets
main_p['label'] = 1
main_n['label'] = 0
validation_p['label'] = 1
validation_n['label'] = 0

# Combine datasets
train_data = pd.concat([main_p, main_n], ignore_index=True)
val_data = pd.concat([validation_p, validation_n], ignore_index=True)

# Separate features and labels
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values
X_val = val_data.drop(columns=['label']).values
y_val = val_data['label'].values


In [ ]:
# Dictionary of models
models = {
    "SVM": SVC(kernel='linear', probability=True),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
    "Neural Network": Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ]),
    "MLP": Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
}


In [ ]:
# Compile the neural network models
models["Neural Network"].compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
models["MLP"].compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Store accuracies
results = []

# Train each model and evaluate
for name, model in models.items():
    print(f"\nTraining {name}...")

    if name in ["Neural Network", "MLP"]:
        # Neural Network training
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), verbose=0)
        train_pred = (model.predict(X_train) > 0.5).astype("int32")
        val_pred = (model.predict(X_val) > 0.5).astype("int32")
    else:
        # Traditional ML model training
        model.fit(X_train, y_train)
        train_pred = model.predict(X_train)
        val_pred = model.predict(X_val)

    # Calculate train and validation accuracy
    train_accuracy = accuracy_score(y_train, train_pred)
    val_accuracy = accuracy_score(y_val, val_pred)

    # Append results
    results.append({"Model": name, "Train Accuracy": train_accuracy, "Validation Accuracy": val_accuracy})



Training SVM...

Training Decision Tree...

Training Random Forest...

Training Logistic Regression...

Training k-NN...

Training Naive Bayes...

Training Gradient Boosting...

Training XGBoost...

Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000690 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 598
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Training CatBoost...

Training AdaBoost...

Training Neural Network...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Training MLP...
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [ ]:
# Convert to DataFrame and sort by Validation Accuracy and Train Accuracy in descending order
results_df = pd.DataFrame(results).sort_values(by=["Validation Accuracy", "Train Accuracy"], ascending=False).reset_index(drop=True)

# Display results
print("\nModel Accuracy Table (Descending Order of Validation Accuracy)")
print(results_df)


Model Accuracy Table (Descending Order of Validation Accuracy)
                  Model  Train Accuracy  Validation Accuracy
0               XGBoost        0.979381             0.876667
1              LightGBM        0.977663             0.863333
2         Random Forest        0.981100             0.856667
3                  k-NN        0.871993             0.850000
4         Decision Tree        0.981100             0.840000
5              CatBoost        0.943299             0.833333
6     Gradient Boosting        0.902921             0.826667
7        Neural Network        0.851375             0.823333
8                   MLP        0.918385             0.816667
9              AdaBoost        0.823024             0.763333
10                  SVM        0.704467             0.703333
11  Logistic Regression        0.687285             0.683333
12          Naive Bayes        0.676976             0.680000


# Code for Cross-Validation

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
# Dictionary of models
models = {
    "SVM": SVC(kernel='linear', probability=True),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
}


In [ ]:
# Define Neural Network models
def create_neural_network(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_mlp(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
# Cross-validation for traditional models
results = []

# For traditional ML models, we use cross_val_score
for name, model in models.items():
    print(f"\nPerforming Cross-validation for {name}...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Calculate cross-validation accuracy
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    mean_accuracy = np.mean(cv_scores)
    std_accuracy = np.std(cv_scores)

    results.append({"Model": name, "Mean CV Accuracy": mean_accuracy, "STD CV Accuracy": std_accuracy})

# Cross-validation for Neural Networks (manual implementation)
for name, create_model in [("Neural Network", create_neural_network), ("MLP", create_mlp)]:
    print(f"\nPerforming Cross-validation for {name}...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Custom function to calculate accuracy for neural networks
    def neural_network_cross_val(model_func, X_train, y_train):
        accuracies = []
        for train_index, val_index in cv.split(X_train, y_train):
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

            model = model_func(X_train.shape[1])  # Create a new model for each fold
            model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=32, verbose=0)

            y_pred = (model.predict(X_val_fold) > 0.5).astype("int32")
            accuracy = accuracy_score(y_val_fold, y_pred)
            accuracies.append(accuracy)

        return np.mean(accuracies), np.std(accuracies)

    mean_accuracy, std_accuracy = neural_network_cross_val(create_model, X_train, y_train)
    results.append({"Model": name, "Mean CV Accuracy": mean_accuracy, "STD CV Accuracy": std_accuracy})



Performing Cross-validation for SVM...

Performing Cross-validation for Decision Tree...

Performing Cross-validation for Random Forest...

Performing Cross-validation for Logistic Regression...

Performing Cross-validation for k-NN...

Performing Cross-validation for Naive Bayes...

Performing Cross-validation for Gradient Boosting...

Performing Cross-validation for XGBoost...

Performing Cross-validation for LightGBM...
[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 552
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Info] Numb

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1


Performing Cross-validation for Neural Network...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

Performing Cross-validation for MLP...
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [ ]:
# Convert to DataFrame and display
cv_results_df = pd.DataFrame(results).sort_values(by="Mean CV Accuracy", ascending=False).reset_index(drop=True)
print("\nCross-Validation Accuracy Table")
print(cv_results_df)


Cross-Validation Accuracy Table
                  Model  Mean CV Accuracy  STD CV Accuracy
0               XGBoost          0.865983         0.011274
1              LightGBM          0.860830         0.009173
2         Random Forest          0.853955         0.003708
3              CatBoost          0.845360         0.006075
4     Gradient Boosting          0.833336         0.017650
5         Decision Tree          0.829040         0.008702
6                   MLP          0.822998         0.019541
7                  k-NN          0.804965         0.016424
8        Neural Network          0.786048         0.021675
9              AdaBoost          0.780032         0.026457
10                  SVM          0.694990         0.016600
11  Logistic Regression          0.687232         0.035055
12          Naive Bayes          0.679521         0.021140


# hyperparameter optimization with Optuna

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install scikeras

In [ ]:
import pandas as pd
import optuna
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)])
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)])

# Split features and labels
X_train = main_data.drop("label", axis=1)
y_train = main_data["label"]
X_val = validation_data.drop("label", axis=1)
y_val = validation_data["label"]



# Define models with MLP included
models = {
    "SVM": lambda trial: SVC(
        C=trial.suggest_float("C", 0.1, 10.0),
        kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
    ),
    "Decision Tree": lambda trial: DecisionTreeClassifier(
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Random Forest": lambda trial: RandomForestClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Logistic Regression": lambda trial: LogisticRegression(
        C=trial.suggest_float("C", 0.1, 10.0),
        solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])
    ),
    "k-NN": lambda trial: KNeighborsClassifier(
        n_neighbors=trial.suggest_int("n_neighbors", 3, 20)
    ),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        max_depth=trial.suggest_int("max_depth", 3, 20)
    ),
    "XGBoost": lambda trial: XGBClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        use_label_encoder=False,
        eval_metric="logloss"
    ),
    "LightGBM": lambda trial: LGBMClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)
    ),
    "AdaBoost": lambda trial: AdaBoostClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)
    ),
    "Neural Network": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("hidden_layer_1", 10, 100),
            trial.suggest_int("hidden_layer_2", 10, 100)
        ),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200
    ),
    "MLP": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("layer_1", 50, 150),
            trial.suggest_int("layer_2", 50, 150)
        ),
        activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]),
        solver=trial.suggest_categorical("solver", ["adam", "sgd"]),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200,
        random_state=42
    )
}


results = []

def optimize_model(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        return accuracy_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Store the results
    results.append({
        "Model": model_name,
        "Accuracy": study.best_value,
        "Best Params": study.best_params
    })

# Run optimization for all models
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    optimize_model(model_name, model_func)


# Convert results to a DataFrame
results_df = pd.DataFrame(results)


# Display the DataFrame
print(results_df)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
[I 2025-01-02 07:34:35,234] A new study created in memory with name: no-name-2cdf44ca-757a-4436-aa34-283560313131
[I 2025-01-02 07:34:35,297] Trial 0 finished with value: 0.7866666666666666 and parameters: {'C': 1.5587163393417423, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7866666666666666.
[I 2025-01-02 07:34:35,371] Trial 1 finished with value: 0.6233333333333333 and parameters: {'C': 4.947158447036206, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.7866666666666666.
[I 2025-01-02 07:34:35,420] Trial 2 finished with value: 0.7033333333333334 and parameters: {'C': 0.33221625597117066, 'kernel': 'linear'}. Best is trial 0 with value: 0.7866666666666666.


Optimizing SVM...


[I 2025-01-02 07:34:35,505] Trial 3 finished with value: 0.7833333333333333 and parameters: {'C': 9.183702801710174, 'kernel': 'poly'}. Best is trial 0 with value: 0.7866666666666666.
[I 2025-01-02 07:34:35,581] Trial 4 finished with value: 0.6333333333333333 and parameters: {'C': 0.3388479693620935, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.7866666666666666.
[I 2025-01-02 07:34:35,655] Trial 5 finished with value: 0.79 and parameters: {'C': 7.430929552803818, 'kernel': 'poly'}. Best is trial 5 with value: 0.79.
[I 2025-01-02 07:34:35,729] Trial 6 finished with value: 0.7766666666666666 and parameters: {'C': 3.429998415859077, 'kernel': 'rbf'}. Best is trial 5 with value: 0.79.
[I 2025-01-02 07:34:35,777] Trial 7 finished with value: 0.73 and parameters: {'C': 8.239638347026967, 'kernel': 'linear'}. Best is trial 5 with value: 0.79.
[I 2025-01-02 07:34:35,838] Trial 8 finished with value: 0.6233333333333333 and parameters: {'C': 5.461408171643127, 'kernel': 'sigmoid'}. Best i

Optimizing Decision Tree...


[I 2025-01-02 07:34:37,732] Trial 11 finished with value: 0.85 and parameters: {'max_depth': 15, 'min_samples_split': 5}. Best is trial 11 with value: 0.85.
[I 2025-01-02 07:34:37,764] Trial 12 finished with value: 0.8266666666666667 and parameters: {'max_depth': 14, 'min_samples_split': 6}. Best is trial 11 with value: 0.85.
[I 2025-01-02 07:34:37,791] Trial 13 finished with value: 0.8466666666666667 and parameters: {'max_depth': 16, 'min_samples_split': 3}. Best is trial 11 with value: 0.85.
[I 2025-01-02 07:34:37,819] Trial 14 finished with value: 0.8433333333333334 and parameters: {'max_depth': 16, 'min_samples_split': 3}. Best is trial 11 with value: 0.85.
[I 2025-01-02 07:34:37,843] Trial 15 finished with value: 0.8366666666666667 and parameters: {'max_depth': 17, 'min_samples_split': 10}. Best is trial 11 with value: 0.85.
[I 2025-01-02 07:34:37,870] Trial 16 finished with value: 0.8366666666666667 and parameters: {'max_depth': 13, 'min_samples_split': 5}. Best is trial 11 with 

...

Optimizing Neural Network...


[I 2025-01-02 07:37:31,001] Trial 0 finished with value: 0.75 and parameters: {'hidden_layer_1': 63, 'hidden_layer_2': 36, 'learning_rate_init': 0.03524606650272839}. Best is trial 0 with value: 0.75.
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-02 07:37:31,882] Trial 1 finished with value: 0.7933333333333333 and parameters: {'hidden_layer_1': 36, 'hidden_layer_2': 15, 'learning_rate_init': 0.010132694609414562}. Best is trial 1 with value: 0.7933333333333333.
[I 2025-01-02 07:37:32,787] Trial 2 finished with value: 0.7866666666666666 and parameters: {'hidden_layer_1': 73, 'hidden_layer_2': 35, 'learning_rate_init': 0.088116520008631}. Best is trial 1 with value: 0.7933333333333333.
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Optimizing MLP...


[I 2025-01-02 07:38:04,000] Trial 0 finished with value: 0.7133333333333334 and parameters: {'layer_1': 64, 'layer_2': 52, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.015444342040856079}. Best is trial 0 with value: 0.7133333333333334.
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-02 07:38:06,868] Trial 1 finished with value: 0.7766666666666666 and parameters: {'layer_1': 60, 'layer_2': 106, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.08864550864066753}. Best is trial 1 with value: 0.7766666666666666.
[I 2025-01-02 07:38:11,242] Trial 2 finished with value: 0.6966666666666667 and parameters: {'layer_1': 117, 'layer_2': 53, 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.014964933328409945}. Best is trial 1 with value: 0.7766666666666

                  Model  Accuracy  \
0                   SVM  0.790000   
1         Decision Tree  0.850000   
2         Random Forest  0.866667   
3   Logistic Regression  0.706667   
4                  k-NN  0.853333   
5           Naive Bayes  0.680000   
6     Gradient Boosting  0.870000   
7               XGBoost  0.886667   
8              LightGBM  0.876667   
9              AdaBoost  0.760000   
10       Neural Network  0.836667   
11                  MLP  0.840000   

                                          Best Params  
0          {'C': 7.430929552803818, 'kernel': 'poly'}  
1           {'max_depth': 15, 'min_samples_split': 5}  
2   {'n_estimators': 434, 'max_depth': 19, 'min_sa...  
3     {'C': 7.596524581447028, 'solver': 'liblinear'}  
4                                  {'n_neighbors': 3}  
5                                                  {}  
6   {'n_estimators': 406, 'learning_rate': 0.18357...  
7   {'n_estimators': 333, 'max_depth': 6, 'learnin...  
8   {'n_estima

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score, roc_auc_score # Import confusion_matrix and other metrics

# Function to calculate metrics with model name
def calculate_metrics(y_true, y_pred, model_name=None):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    mcc = matthews_corrcoef(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    return {
        "Model": model_name,
        "Accuracy": accuracy,
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "MCC": mcc,
        "Kappa": kappa,
        "AUC": auc,
    }

# Results storage
results = []

# Optimization function
def optimize_model_with_metrics(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        trial.set_user_attr("preds", preds)
        metrics = calculate_metrics(y_val, preds, model_name=model_name)
        return metrics["Accuracy"]

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Store the best trial metrics
    best_trial_preds = study.best_trial.user_attrs["preds"]
    best_metrics = calculate_metrics(y_val, best_trial_preds, model_name=model_name)
    best_metrics["Best Params"] = study.best_params
    results.append(best_metrics)

# Run optimization for all models
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    optimize_model_with_metrics(model_name, model_func)

# Store the final results with selected metrics and best parameters
final_results = []
for result in results:
    final_results.append(
        {
            "Model": result["Model"],
            "Accuracy": result["Accuracy"],
            "Sensitivity": result["Sensitivity"],
            "Specificity": result["Specificity"],
            "MCC": result["MCC"],
            "Kappa": result["Kappa"],
            "AUC": result["AUC"],
            "Best Params": result["Best Params"],
        }
    )

# Convert results to a DataFrame and display it
final_results_df = pd.DataFrame(final_results)
print(final_results_df)


[I 2025-01-02 07:39:39,532] A new study created in memory with name: no-name-08d7851b-85a1-4cda-a43a-0dcefd3eb883
[I 2025-01-02 07:39:39,601] Trial 0 finished with value: 0.6266666666666667 and parameters: {'C': 6.717374225234966, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.6266666666666667.
[I 2025-01-02 07:39:39,684] Trial 1 finished with value: 0.6166666666666667 and parameters: {'C': 2.8225806330375063, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.6266666666666667.


Optimizing SVM...


[I 2025-01-02 07:39:39,754] Trial 2 finished with value: 0.62 and parameters: {'C': 7.7116843707125176, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.6266666666666667.
[I 2025-01-02 07:39:39,809] Trial 3 finished with value: 0.6933333333333334 and parameters: {'C': 0.14732256675530456, 'kernel': 'linear'}. Best is trial 3 with value: 0.6933333333333334.
[I 2025-01-02 07:39:39,874] Trial 4 finished with value: 0.7833333333333333 and parameters: {'C': 3.016066253630342, 'kernel': 'poly'}. Best is trial 4 with value: 0.7833333333333333.
[I 2025-01-02 07:39:39,928] Trial 5 finished with value: 0.73 and parameters: {'C': 8.073220604081383, 'kernel': 'linear'}. Best is trial 4 with value: 0.7833333333333333.
[I 2025-01-02 07:39:39,994] Trial 6 finished with value: 0.7866666666666666 and parameters: {'C': 6.156471117558458, 'kernel': 'rbf'}. Best is trial 6 with value: 0.7866666666666666.
[I 2025-01-02 07:39:40,051] Trial 7 finished with value: 0.73 and parameters: {'C': 7.9982842345332

Optimizing Decision Tree...


[I 2025-01-02 07:39:42,220] Trial 7 finished with value: 0.84 and parameters: {'max_depth': 18, 'min_samples_split': 3}. Best is trial 1 with value: 0.8466666666666667.
[I 2025-01-02 07:39:42,244] Trial 8 finished with value: 0.83 and parameters: {'max_depth': 19, 'min_samples_split': 7}. Best is trial 1 with value: 0.8466666666666667.
[I 2025-01-02 07:39:42,275] Trial 9 finished with value: 0.8266666666666667 and parameters: {'max_depth': 7, 'min_samples_split': 8}. Best is trial 1 with value: 0.8466666666666667.
[I 2025-01-02 07:39:42,334] Trial 10 finished with value: 0.8666666666666667 and parameters: {'max_depth': 11, 'min_samples_split': 2}. Best is trial 10 with value: 0.8666666666666667.
[I 2025-01-02 07:39:42,378] Trial 11 finished with value: 0.81 and parameters: {'max_depth': 11, 'min_samples_split': 10}. Best is trial 10 with value: 0.8666666666666667.
[I 2025-01-02 07:39:42,421] Trial 12 finished with value: 0.8566666666666667 and parameters: {'max_depth': 11, 'min_samples

...

Optimizing Neural Network...


[I 2025-01-02 07:42:40,917] Trial 0 finished with value: 0.7966666666666666 and parameters: {'hidden_layer_1': 35, 'hidden_layer_2': 17, 'learning_rate_init': 0.019642110483302454}. Best is trial 0 with value: 0.7966666666666666.
[I 2025-01-02 07:42:43,780] Trial 1 finished with value: 0.77 and parameters: {'hidden_layer_1': 45, 'hidden_layer_2': 61, 'learning_rate_init': 0.01648006148294367}. Best is trial 0 with value: 0.7966666666666666.
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-02 07:42:44,840] Trial 2 finished with value: 0.7766666666666666 and parameters: {'hidden_layer_1': 10, 'hidden_layer_2': 20, 'learning_rate_init': 0.013716018794592645}. Best is trial 0 with value: 0.7966666666666666.
[I 2025-01-02 07:42:45,665] Trial 3 finished with value: 0.81 and parameters: {'hidden_layer_1':

Optimizing MLP...


[I 2025-01-02 07:43:32,163] Trial 0 finished with value: 0.8166666666666667 and parameters: {'layer_1': 63, 'layer_2': 106, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.04365631190271952}. Best is trial 0 with value: 0.8166666666666667.
[I 2025-01-02 07:43:32,774] Trial 1 finished with value: 0.5 and parameters: {'layer_1': 76, 'layer_2': 121, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.08925264091097065}. Best is trial 0 with value: 0.8166666666666667.
[I 2025-01-02 07:43:34,628] Trial 2 finished with value: 0.7533333333333333 and parameters: {'layer_1': 58, 'layer_2': 139, 'activation': 'relu', 'solver': 'sgd', 'learning_rate_init': 0.08242562471049965}. Best is trial 0 with value: 0.8166666666666667.
[I 2025-01-02 07:43:35,629] Trial 3 finished with value: 0.77 and parameters: {'layer_1': 92, 'layer_2': 55, 'activation': 'relu', 'solver': 'adam', 'learning_rate_init': 0.08288551283410249}. Best is trial 0 with value: 0.8166666666666667.
[I 2

                  Model  Accuracy  Sensitivity  Specificity       MCC  \
0                   SVM  0.790000     0.713333     0.866667  0.586941   
1         Decision Tree  0.866667     0.853333     0.880000  0.733594   
2         Random Forest  0.866667     0.840000     0.893333  0.734379   
3   Logistic Regression  0.706667     0.680000     0.733333  0.413922   
4                  k-NN  0.853333     0.840000     0.866667  0.706918   
5           Naive Bayes  0.680000     0.660000     0.700000  0.360288   
6     Gradient Boosting  0.873333     0.860000     0.886667  0.746932   
7               XGBoost  0.880000     0.860000     0.900000  0.760609   
8              LightGBM  0.880000     0.853333     0.906667  0.761083   
9              AdaBoost  0.763333     0.726667     0.800000  0.528089   
10       Neural Network  0.840000     0.793333     0.886667  0.682981   
11                  MLP  0.850000     0.833333     0.866667  0.700389   

       Kappa       AUC                            

In [ ]:
import pandas as pd
import optuna
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Define models
models = {
    "SVM": lambda trial: SVC(probability=True, C=trial.suggest_float("C", 0.1, 10.0), kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])),
    "Decision Tree": lambda trial: DecisionTreeClassifier(max_depth=trial.suggest_int("max_depth", 3, 20), min_samples_split=trial.suggest_int("min_samples_split", 2, 10)),
    "Random Forest": lambda trial: RandomForestClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), min_samples_split=trial.suggest_int("min_samples_split", 2, 10)),
    "Logistic Regression": lambda trial: LogisticRegression(C=trial.suggest_float("C", 0.1, 10.0), solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])),
    "k-NN": lambda trial: KNeighborsClassifier(n_neighbors=trial.suggest_int("n_neighbors", 3, 20)),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5), max_depth=trial.suggest_int("max_depth", 3, 20)),
    "XGBoost": lambda trial: XGBClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5), use_label_encoder=False, eval_metric="logloss"),
    "LightGBM": lambda trial: LGBMClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)),
    "AdaBoost": lambda trial: AdaBoostClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)),
    "Neural Network": lambda trial: MLPClassifier(hidden_layer_sizes=(trial.suggest_int("hidden_layer_1", 10, 100), trial.suggest_int("hidden_layer_2", 10, 100)), learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1), max_iter=200),
    "MLP": lambda trial: MLPClassifier(hidden_layer_sizes=(trial.suggest_int("layer_1", 50, 150), trial.suggest_int("layer_2", 50, 150)), activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]), solver=trial.suggest_categorical("solver", ["adam", "sgd"]), learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1), max_iter=200, random_state=42)
}

# Prepare a dictionary to store model probabilities horizontally
probabilities = {"Target": y_val}  # Starting with the target column (y_val)

# Run optimization and compute probabilities for each model
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    study = optuna.create_study(direction="maximize")

    # Objective function for Optuna
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        return accuracy_score(y_val, model.predict(X_val))

    study.optimize(objective, n_trials=30)

    # Train the best model using the best hyperparameters
    best_model = model_func(study.best_trial)
    best_model.fit(X_train, y_train)

    # Get predicted probabilities for the positive class (class 1)
    probs = best_model.predict_proba(X_val)[:, 1]

    # Add to the probabilities dictionary
    probabilities[model_name] = probs

# Convert the probabilities dictionary to a DataFrame
probability_df = pd.DataFrame(probabilities)

# Save the probability dataset to a CSV file
probability_df.to_csv("N_GDC_OPTUNA_probability_predictions.csv", index=False)

print("Dataset saved successfully!")


[I 2025-01-02 07:45:04,782] A new study created in memory with name: no-name-060fe927-db82-4829-9670-0abae4329b95


Optimizing SVM...


[I 2025-01-02 07:45:05,390] Trial 0 finished with value: 0.78 and parameters: {'C': 9.435187062312181, 'kernel': 'poly'}. Best is trial 0 with value: 0.78.
[I 2025-01-02 07:45:05,625] Trial 1 finished with value: 0.78 and parameters: {'C': 5.402191374444786, 'kernel': 'rbf'}. Best is trial 0 with value: 0.78.
[I 2025-01-02 07:45:05,858] Trial 2 finished with value: 0.62 and parameters: {'C': 8.33048583802585, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.78.
[I 2025-01-02 07:45:06,027] Trial 3 finished with value: 0.7366666666666667 and parameters: {'C': 9.517428581832885, 'kernel': 'linear'}. Best is trial 0 with value: 0.78.
[I 2025-01-02 07:45:06,195] Trial 4 finished with value: 0.71 and parameters: {'C': 2.0784494759671523, 'kernel': 'linear'}. Best is trial 0 with value: 0.78.
[I 2025-01-02 07:45:06,490] Trial 5 finished with value: 0.79 and parameters: {'C': 5.094289375863873, 'kernel': 'poly'}. Best is trial 5 with value: 0.79.
[I 2025-01-02 07:45:06,756] Trial 6 finished

Optimizing Decision Tree...


[I 2025-01-02 07:45:13,332] Trial 12 finished with value: 0.8533333333333334 and parameters: {'max_depth': 20, 'min_samples_split': 2}. Best is trial 11 with value: 0.8533333333333334.
[I 2025-01-02 07:45:13,368] Trial 13 finished with value: 0.85 and parameters: {'max_depth': 13, 'min_samples_split': 2}. Best is trial 11 with value: 0.8533333333333334.
[I 2025-01-02 07:45:13,393] Trial 14 finished with value: 0.85 and parameters: {'max_depth': 20, 'min_samples_split': 3}. Best is trial 11 with value: 0.8533333333333334.
[I 2025-01-02 07:45:13,419] Trial 15 finished with value: 0.86 and parameters: {'max_depth': 13, 'min_samples_split': 3}. Best is trial 15 with value: 0.86.
[I 2025-01-02 07:45:13,444] Trial 16 finished with value: 0.8433333333333334 and parameters: {'max_depth': 12, 'min_samples_split': 4}. Best is trial 15 with value: 0.86.
[I 2025-01-02 07:45:13,469] Trial 17 finished with value: 0.83 and parameters: {'max_depth': 9, 'min_samples_split': 3}. Best is trial 15 with va

...

[I 2025-01-02 07:48:01,219] Trial 0 finished with value: 0.8 and parameters: {'hidden_layer_1': 48, 'hidden_layer_2': 77, 'learning_rate_init': 0.017188045534543082}. Best is trial 0 with value: 0.8.
[I 2025-01-02 07:48:01,723] Trial 1 finished with value: 0.7233333333333334 and parameters: {'hidden_layer_1': 15, 'hidden_layer_2': 43, 'learning_rate_init': 0.09324404520329169}. Best is trial 0 with value: 0.8.
[I 2025-01-02 07:48:02,673] Trial 2 finished with value: 0.7433333333333333 and parameters: {'hidden_layer_1': 28, 'hidden_layer_2': 83, 'learning_rate_init': 0.0905217768494729}. Best is trial 0 with value: 0.8.
[I 2025-01-02 07:48:03,804] Trial 3 finished with value: 0.7633333333333333 and parameters: {'hidden_layer_1': 38, 'hidden_layer_2': 66, 'learning_rate_init': 0.0860274652392535}. Best is trial 0 with value: 0.8.
[I 2025-01-02 07:48:05,419] Trial 4 finished with value: 0.7666666666666667 and parameters: {'hidden_layer_1': 86, 'hidden_layer_2': 33, 'learning_rate_init': 0

Optimizing MLP...


[I 2025-01-02 07:48:46,060] Trial 0 finished with value: 0.7666666666666667 and parameters: {'layer_1': 54, 'layer_2': 128, 'activation': 'relu', 'solver': 'adam', 'learning_rate_init': 0.07080354687620669}. Best is trial 0 with value: 0.7666666666666667.
[I 2025-01-02 07:48:47,203] Trial 1 finished with value: 0.7666666666666667 and parameters: {'layer_1': 110, 'layer_2': 92, 'activation': 'relu', 'solver': 'adam', 'learning_rate_init': 0.09446692243342501}. Best is trial 0 with value: 0.7666666666666667.
[I 2025-01-02 07:48:47,786] Trial 2 finished with value: 0.7133333333333334 and parameters: {'layer_1': 63, 'layer_2': 63, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.09420947307452292}. Best is trial 0 with value: 0.7666666666666667.
[I 2025-01-02 07:48:50,029] Trial 3 finished with value: 0.8266666666666667 and parameters: {'layer_1': 123, 'layer_2': 64, 'activation': 'relu', 'solver': 'adam', 'learning_rate_init': 0.06545938076796519}. Best is trial 3 with valu

Dataset saved successfully!


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Class Feature Vector (CFV)

In [ ]:
import optuna
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)])
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)])

# Split features and labels
X_train = main_data.drop("label", axis=1)
y_train = main_data["label"]
X_val = validation_data.drop("label", axis=1)
y_val = validation_data["label"]

# Define models with hyperparameter optimization (Optuna)
models = {
    "SVM": lambda trial: SVC(
         probability=True,
        C=trial.suggest_float("C", 0.1, 10.0),
        kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
    ),
    "Decision Tree": lambda trial: DecisionTreeClassifier(
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Random Forest": lambda trial: RandomForestClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Logistic Regression": lambda trial: LogisticRegression(
        C=trial.suggest_float("C", 0.1, 10.0),
        solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])
    ),
    "k-NN": lambda trial: KNeighborsClassifier(
        n_neighbors=trial.suggest_int("n_neighbors", 3, 20)
    ),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        max_depth=trial.suggest_int("max_depth", 3, 20)
    ),
    "XGBoost": lambda trial: XGBClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        use_label_encoder=False,
        eval_metric="logloss"
    ),
    "LightGBM": lambda trial: LGBMClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)
    ),
    "AdaBoost": lambda trial: AdaBoostClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)
    ),
    "Neural Network": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("hidden_layer_1", 10, 100),
            trial.suggest_int("hidden_layer_2", 10, 100)
        ),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200
    ),
    "MLP": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("layer_1", 50, 150),
            trial.suggest_int("layer_2", 50, 150)
        ),
        activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]),
        solver=trial.suggest_categorical("solver", ["adam", "sgd"]),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200,
        random_state=42
    )
}

# Initialize a list to store the CFV data
cfv_data = []

# Define the optimization and prediction function
def optimize_and_predict(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_val)[:, 1]  # Get probability for class 1 (positive)
        return accuracy_score(y_val, model.predict(X_val))

    # Perform optimization with Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Return the best model
    best_model = model_func(study.best_trial)
    best_model.fit(X_train, y_train)
    preds = best_model.predict_proba(X_val)[:, 1]  # Using the probability for class 1 (positive)

    # Append predictions to CFV list
    return preds

# Train each model and generate predictions for CFV
for model_name, model_func in models.items():
    print(f"Training and predicting with {model_name}...")
    preds = optimize_and_predict(model_name, model_func)
    cfv_data.append(preds)

# Convert the CFV data into a DataFrame
cfv_df = pd.DataFrame(np.array(cfv_data).T, columns=models.keys())

# Optionally, add the true labels column
cfv_df["True_Label"] = y_val.values

# Save the CFV dataset to CSV
cfv_df.to_csv("CFV_GDC.csv", index=False)
print("CFV dataset created and saved!")


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
[I 2025-01-16 03:42:52,545] A new study created in memory with name: no-name-5bd4801d-c6a6-4cb8-888a-4f534057a7e7


Training and predicting with SVM...


[I 2025-01-16 03:42:52,806] Trial 0 finished with value: 0.7833333333333333 and parameters: {'C': 4.374651706952948, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7833333333333333.
[I 2025-01-16 03:42:53,048] Trial 1 finished with value: 0.7833333333333333 and parameters: {'C': 4.58689453320545, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7833333333333333.
[I 2025-01-16 03:42:53,296] Trial 2 finished with value: 0.78 and parameters: {'C': 3.3710181330853315, 'kernel': 'poly'}. Best is trial 0 with value: 0.7833333333333333.
[I 2025-01-16 03:42:53,602] Trial 3 finished with value: 0.7866666666666666 and parameters: {'C': 6.504792719554885, 'kernel': 'poly'}. Best is trial 3 with value: 0.7866666666666666.
[I 2025-01-16 03:42:53,836] Trial 4 finished with value: 0.7533333333333333 and parameters: {'C': 0.4787956273302435, 'kernel': 'rbf'}. Best is trial 3 with value: 0.7866666666666666.
[I 2025-01-16 03:42:54,087] Trial 5 finished with value: 0.62 and parameters: {'C': 8.3468823620

Training and predicting with Decision Tree...


[I 2025-01-16 03:43:01,559] Trial 12 finished with value: 0.85 and parameters: {'max_depth': 17, 'min_samples_split': 4}. Best is trial 11 with value: 0.8566666666666667.
[I 2025-01-16 03:43:01,585] Trial 13 finished with value: 0.84 and parameters: {'max_depth': 17, 'min_samples_split': 4}. Best is trial 11 with value: 0.8566666666666667.
[I 2025-01-16 03:43:01,610] Trial 14 finished with value: 0.8266666666666667 and parameters: {'max_depth': 18, 'min_samples_split': 4}. Best is trial 11 with value: 0.8566666666666667.
[I 2025-01-16 03:43:01,635] Trial 15 finished with value: 0.8366666666666667 and parameters: {'max_depth': 9, 'min_samples_split': 3}. Best is trial 11 with value: 0.8566666666666667.
[I 2025-01-16 03:43:01,660] Trial 16 finished with value: 0.8333333333333334 and parameters: {'max_depth': 9, 'min_samples_split': 4}. Best is trial 11 with value: 0.8566666666666667.
[I 2025-01-16 03:43:01,686] Trial 17 finished with value: 0.8566666666666667 and parameters: {'max_depth'

...

Training and predicting with Neural Network...


[I 2025-01-16 03:45:40,436] Trial 0 finished with value: 0.8033333333333333 and parameters: {'hidden_layer_1': 62, 'hidden_layer_2': 60, 'learning_rate_init': 0.08224088138725932}. Best is trial 0 with value: 0.8033333333333333.
[I 2025-01-16 03:45:41,335] Trial 1 finished with value: 0.7966666666666666 and parameters: {'hidden_layer_1': 26, 'hidden_layer_2': 99, 'learning_rate_init': 0.05833172345210263}. Best is trial 0 with value: 0.8033333333333333.
[I 2025-01-16 03:45:41,873] Trial 2 finished with value: 0.7966666666666666 and parameters: {'hidden_layer_1': 63, 'hidden_layer_2': 62, 'learning_rate_init': 0.0748348403102436}. Best is trial 0 with value: 0.8033333333333333.
[I 2025-01-16 03:45:43,245] Trial 3 finished with value: 0.8133333333333334 and parameters: {'hidden_layer_1': 98, 'hidden_layer_2': 74, 'learning_rate_init': 0.021630466402413503}. Best is trial 3 with value: 0.8133333333333334.
[I 2025-01-16 03:45:45,215] Trial 4 finished with value: 0.8066666666666666 and para

Training and predicting with MLP...


[I 2025-01-16 03:46:27,862] Trial 0 finished with value: 0.5 and parameters: {'layer_1': 150, 'layer_2': 142, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.08039020041500193}. Best is trial 0 with value: 0.5.
[I 2025-01-16 03:46:29,717] Trial 1 finished with value: 0.7133333333333334 and parameters: {'layer_1': 52, 'layer_2': 65, 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.041531712880350846}. Best is trial 1 with value: 0.7133333333333334.
[I 2025-01-16 03:46:29,991] Trial 2 finished with value: 0.5 and parameters: {'layer_1': 112, 'layer_2': 88, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.08164991180756852}. Best is trial 1 with value: 0.7133333333333334.
[I 2025-01-16 03:46:31,096] Trial 3 finished with value: 0.7066666666666667 and parameters: {'layer_1': 68, 'layer_2': 56, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.004714162155213947}. Best is trial 1 with value: 0.7133333333333334.
[I 2025-01-16 

CFV dataset created and saved!


CPFV (Combined Probability and Class Feature Vector)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier


# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)], ignore_index=True)
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)], ignore_index=True)

# Separate features and labels
X_train = main_data.drop(columns=["label"])
y_train = main_data["label"]
X_val = validation_data.drop(columns=["label"])
y_val = validation_data["label"]

# Initialize models with their tuned hyperparameters
trained_models = {
    "SVM": SVC(C=1.0, kernel="rbf", probability=True),  # Example parameters
    "Decision Tree": DecisionTreeClassifier(max_depth=10, min_samples_split=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5),
    "Logistic Regression": LogisticRegression(C=1.0, solver="lbfgs"),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=10),
    "XGBoost": XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1, use_label_encoder=False, eval_metric="logloss"),
    "LightGBM": LGBMClassifier(n_estimators=100, max_depth=10, learning_rate=0.1),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, learning_rate=0.1),
    "Neural Network (MLPClassifier)": MLPClassifier(hidden_layer_sizes=(100, 50), activation="relu", solver="adam", learning_rate_init=0.01, max_iter=200),
    "Multilayer Perceptron (Custom MLP)": MLPClassifier(hidden_layer_sizes=(128, 64), activation="relu", solver="adam", learning_rate_init=0.01, max_iter=200)
}

# Train all models on the training dataset
for model_name, model in trained_models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)

# Function to create CPFV dataset
def create_cpfv(models, X_data, y_data):
    cpfv_data = pd.DataFrame()
    for model_name, model in models.items():
        # Add predicted class labels
        cpfv_data[f"{model_name}_Class"] = model.predict(X_data)
        # Add predicted probabilities or decision scores
        if hasattr(model, "predict_proba"):
            cpfv_data[f"{model_name}_Prob"] = model.predict_proba(X_data)[:, 1]
        elif hasattr(model, "decision_function"):
            cpfv_data[f"{model_name}_Prob"] = model.decision_function(X_data)
        else:
            cpfv_data[f"{model_name}_Prob"] = cpfv_data[f"{model_name}_Class"]
    # Add true labels
    cpfv_data["True_Label"] = y_data.reset_index(drop=True)
    return cpfv_data

# Create CPFV dataset using validation data
cpfv_dataset = create_cpfv(trained_models, X_val, y_val)

# Save CPFV dataset to CSV
cpfv_dataset.to_csv("CPFV_GDC.csv", index=False)

Training SVM...
Training Decision Tree...
Training Random Forest...
Training Logistic Regression...
Training k-NN...
Training Naive Bayes...
Training Gradient Boosting...
Training XGBoost...
Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000166 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 598
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:48:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training AdaBoost...
Training Neural Network (MLPClassifier)...
Training Multilayer Perceptron (Custom MLP)...


grids for RandomizedSearchCV

In [ ]:
#import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier # Import path for KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Combine positive and negative samples
X_train = pd.concat([main_p, main_n])
y_train = np.concatenate([np.ones(len(main_p)), np.zeros(len(main_n))])

# Define cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Function to create a neural network model
def create_nn(num_units, dropout_rate, learning_rate, input_shape):
    model = Sequential([
        Dense(num_units, activation='relu', input_shape=input_shape),
        Dropout(dropout_rate),
        Dense(num_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model definitions and parameter grids for RandomizedSearchCV
models = {
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
    "Neural Network": KerasClassifier(
        model=create_nn,
        num_units=64,
        dropout_rate=0.2,
        learning_rate=0.001,
        input_shape=(X_train.shape[1],),
        epochs=5,
        batch_size=32,
        verbose=0
    )
}

# Parameter grids for each model
param_grids = {
    "SVM": {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']},
    "Decision Tree": {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    "Random Forest": {'n_estimators': [100, 200, 500], 'max_depth': [10, 20, None], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5, 10]},
    "Logistic Regression": {'C': [0.1, 1, 10, 100], 'solver': ['liblinear', 'saga'], 'penalty': ['l2']},
    "k-NN": {'n_neighbors': [3, 5, 11, 19], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']},
    "Naive Bayes": {'var_smoothing': np.logspace(-9, -1, 10)},
    "Gradient Boosting": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    "XGBoost": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    "LightGBM": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [10, 20, -1]},
    "CatBoost": {'depth': [6, 8, 10], 'learning_rate': [0.01, 0.1, 0.2], 'iterations': [100, 200]},
    "AdaBoost": {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1.0]},
    "Neural Network": {
        'model__num_units': [32, 64, 128],
        'model__dropout_rate': [0.1, 0.2, 0.3],
        'model__learning_rate': [0.001, 0.01, 0.1]
    }
}

# Results storage
best_params = {}
best_scores = []

# Loop through models and apply random search
for model_name, model in models.items():
    print(f"Performing RandomizedSearchCV for {model_name}...")
    param_grid = param_grids[model_name]

    # Perform randomized search
    random_search = RandomizedSearchCV(model, param_grid, n_iter=10, cv=cv, scoring='accuracy', n_jobs=-1, random_state=42)
    random_search.fit(X_train, y_train)

    # Store best parameters and score
    best_params[model_name] = random_search.best_params_
    best_scores.append(random_search.best_score_)

# Display results in a DataFrame
results_df = pd.DataFrame({
    'Model': list(models.keys()),
    'Best Score': best_scores,
    'Best Parameters': [best_params[model] for model in models]
})

print(results_df)


Performing RandomizedSearchCV for SVM...
Performing RandomizedSearchCV for Decision Tree...
Performing RandomizedSearchCV for Random Forest...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

Performing RandomizedSearchCV for Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Performing RandomizedSearchCV for k-NN...
Performing RandomizedSearchCV for Naive Bayes...
Performing RandomizedSearchCV for Gradient Boosting...
Performing RandomizedSearchCV for XGBoost...
Performing RandomizedSearchCV for LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000189 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 598
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Performing RandomizedSearchCV for Neural Network...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


                  Model  Best Score  \
0                   SVM    0.792079   
1         Decision Tree    0.831638   
2         Random Forest    0.853974   
3   Logistic Regression    0.719032   
4                  k-NN    0.847081   
5           Naive Bayes    0.683824   
6     Gradient Boosting    0.858254   
7               XGBoost    0.860837   
8              LightGBM    0.859113   
9              CatBoost    0.862561   
10             AdaBoost    0.795471   
11       Neural Network    0.709564   

                                      Best Parameters  
0        {'kernel': 'rbf', 'gamma': 'scale', 'C': 10}  
1   {'min_samples_split': 2, 'min_samples_leaf': 2...  
2   {'n_estimators': 200, 'min_samples_split': 5, ...  
3   {'solver': 'liblinear', 'penalty': 'l2', 'C': ...  
4   {'weights': 'distance', 'n_neighbors': 3, 'met...  
5             {'var_smoothing': 0.012915496650148827}  
6   {'n_estimators': 200, 'max_depth': 7, 'learnin...  
7   {'n_estimators': 200, 'max_depth': 7, 'l

In [ ]:
# Storage for predictions and target column
probability_datasets = pd.DataFrame(y_train, columns=['Target'])

# Loop through models, perform random search, and save probabilities
for model_name, model in models.items():
    print(f"Performing RandomizedSearchCV for {model_name}...")
    param_grid = param_grids[model_name]

    # Perform randomized search
    random_search = RandomizedSearchCV(model,
                                       param_grid,
                                       n_iter=10,
                                       cv=cv,
                                       scoring='accuracy',
                                       n_jobs=-1,
                                       random_state=42)

    random_search.fit(X_train, y_train)

    # Store best parameters and score
    best_params[model_name] = random_search.best_params_
    best_scores.append(random_search.best_score_)



    # Get probability predictions (if supported)
    if hasattr(random_search.best_estimator_, "predict_proba"):
        probabilities = random_search.best_estimator_.predict_proba(X_train)[:, 1]  # Probability for the positive class
        probability_datasets[f"{model_name}_Probabilities"] = probabilities
    else:
        # Fallback if probability prediction isn't supported
        predictions = random_search.best_estimator_.predict(X_train)
        probability_datasets[f"{model_name}_Predictions"] = predictions



# Display final dataset with probabilities
print(probability_datasets.head())

# Save the probability dataset to a CSV file
probability_datasets.to_csv("model_probabilities_with_target GDC in Randomsearch .csv", index=False)
print("Probability dataset saved to 'model_probabilities_with_target.csv'.")


Performing RandomizedSearchCV for SVM...
Performing RandomizedSearchCV for Decision Tree...
Performing RandomizedSearchCV for Random Forest...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

Performing RandomizedSearchCV for Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Performing RandomizedSearchCV for k-NN...
Performing RandomizedSearchCV for Naive Bayes...
Performing RandomizedSearchCV for Gradient Boosting...
Performing RandomizedSearchCV for XGBoost...
Performing RandomizedSearchCV for LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 598
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Performing RandomizedSearchCV for Neural Network...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


   Target  SVM_Predictions  Decision Tree_Probabilities  \
0     1.0              0.0                     1.000000   
1     1.0              1.0                     0.973684   
2     1.0              1.0                     1.000000   
3     1.0              0.0                     0.666667   
4     1.0              1.0                     0.973684   

   Random Forest_Probabilities  Logistic Regression_Probabilities  \
0                     0.997857                           0.317050   
1                     0.972599                           0.990989   
2                     0.997500                           0.972281   
3                     0.725478                           0.473788   
4                     0.972599                           0.990989   

   k-NN_Probabilities  Naive Bayes_Probabilities  \
0                 1.0                   0.262839   
1                 1.0                   1.000000   
2                 1.0                   1.000000   
3                 1.0 

# parameter grids for GridSearchCV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Combine positive and negative samples
X_train = pd.concat([main_p, main_n])
y_train = np.concatenate([np.ones(len(main_p)), np.zeros(len(main_n))])

# Define cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Function to create a neural network model
def create_nn(num_units, dropout_rate, learning_rate, input_shape):
    model = Sequential([
        Dense(num_units, activation='relu', input_shape=input_shape),
        Dropout(dropout_rate),
        Dense(num_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model definitions and parameter grids for GridSearchCV
models = {
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
    "Neural Network": KerasClassifier(
        model=create_nn,
        num_units=64,
        dropout_rate=0.2,
        learning_rate=0.001,
        input_shape=(X_train.shape[1],),
        epochs=5,
        batch_size=32,
        verbose=0
    )
}

# Parameter grids for each model
param_grids = {
    "SVM": {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']},
    "Decision Tree": {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    "Random Forest": {'n_estimators': [100, 200, 500], 'max_depth': [10, 20, None], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5, 10]},
    "Logistic Regression": {'C': [0.1, 1, 10, 100], 'solver': ['liblinear', 'saga'], 'penalty': ['l2']},
    "k-NN": {'n_neighbors': [3, 5, 11, 19], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']},
    "Naive Bayes": {'var_smoothing': np.logspace(-9, -1, 10)},
    "Gradient Boosting": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    "XGBoost": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    "LightGBM": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [10, 20, -1]},
    "CatBoost": {'depth': [6, 8, 10], 'learning_rate': [0.01, 0.1, 0.2], 'iterations': [100, 200]},
    "AdaBoost": {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1.0]},
    "Neural Network": {
        'model__num_units': [32, 64, 128],
        'model__dropout_rate': [0.1, 0.2, 0.3],
        'model__learning_rate': [0.001, 0.01, 0.1]
    }
}

# Results storage
best_params = {}
best_scores = []

# Loop through models and apply grid search
for model_name, model in models.items():
    print(f"Performing GridSearchCV for {model_name}...")
    param_grid = param_grids[model_name]

    # Perform grid search
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Store best parameters and score
    best_params[model_name] = grid_search.best_params_
    best_scores.append(grid_search.best_score_)

# Display results in a DataFrame
results_df = pd.DataFrame({
    'Model': list(models.keys()),
    'Best Score': best_scores,
    'Best Parameters': [best_params[model] for model in models]
})

print(results_df)


Performing GridSearchCV for SVM...
Performing GridSearchCV for Decision Tree...
Performing GridSearchCV for Random Forest...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
135 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", 

Performing GridSearchCV for Logistic Regression...
Performing GridSearchCV for k-NN...
Performing GridSearchCV for Naive Bayes...
Performing GridSearchCV for Gradient Boosting...
Performing GridSearchCV for XGBoost...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Performing GridSearchCV for LightGBM...


...

[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000175 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 598
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Performing GridSearchCV for CatBoost...
Performing GridSearchCV for AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Performing GridSearchCV for Neural Network...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


                  Model  Best Score  \
0                   SVM    0.807533   
1         Decision Tree    0.829051   
2         Random Forest    0.859120   
3   Logistic Regression    0.719032   
4                  k-NN    0.847081   
5           Naive Bayes    0.683824   
6     Gradient Boosting    0.857407   
7               XGBoost    0.860837   
8              LightGBM    0.860830   
9              CatBoost    0.862561   
10             AdaBoost    0.795471   
11       Neural Network    0.717341   

                                      Best Parameters  
0      {'C': 100, 'gamma': 'scale', 'kernel': 'poly'}  
1   {'max_depth': 30, 'min_samples_leaf': 2, 'min_...  
2   {'max_depth': None, 'max_features': 'sqrt', 'm...  
3   {'C': 100, 'penalty': 'l2', 'solver': 'libline...  
4   {'metric': 'manhattan', 'n_neighbors': 3, 'wei...  
5             {'var_smoothing': 0.012915496650148827}  
6   {'learning_rate': 0.1, 'max_depth': 7, 'n_esti...  
7   {'learning_rate': 0.1, 'max_depth': 7, '

In [ ]:
# Prepare the final dataset with probabilities and target
all_probabilities = []
all_targets = []

# Loop through models and apply grid search
for model_name, model in models.items():
    print(f"Performing GridSearchCV for {model_name}...")

    # Special handling for SVC: enable probability estimation
    if model_name == "SVM":
        model.probability = True  # Enable probability for SVC

    # Get the parameter grid for the current model
    param_grid = param_grids[model_name]

    # Perform grid search
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Store best parameters and score
    best_params[model_name] = grid_search.best_params_
    best_scores.append(grid_search.best_score_)

    # Predict probabilities using the best estimator
    best_model = grid_search.best_estimator_
    if hasattr(best_model, "predict_proba"):
        probabilities = best_model.predict_proba(X_train)[:, 1]  # Positive class probabilities
    else:
        # Fallback for models without predict_proba (e.g., SVM with linear kernel)
        probabilities = best_model.decision_function(X_train)
        probabilities = (probabilities - probabilities.min()) / (probabilities.max() - probabilities.min())

    # Append probabilities and targets for this model
    all_probabilities.append(probabilities)
    all_targets.append(y_train)

    # Combine probabilities, features, and target into a DataFrame
    model_data = pd.DataFrame(X_train, columns=main_p.columns)  # Ensure column consistency
    model_data[f"{model_name}_probability"] = probabilities
    model_data['target'] = y_train

    # Save to CSV
    output_path = f"/content/{model_name}_probabilities.csv"
    model_data.to_csv(output_path, index=False)
    print(f"Saved probabilities for {model_name} to {output_path}")

# Combine all model probabilities into a single DataFrame (optional)
final_dataset = pd.DataFrame({'target': y_train})
for idx, model_name in enumerate(models.keys()):
    final_dataset[f"{model_name}_probability"] = all_probabilities[idx]

# Save the combined dataset
final_output_path = "/content/combined_probabilities_GridSearchCV.csv"
final_dataset.to_csv(final_output_path, index=False)
print(f"Saved combined dataset to {final_output_path}")


Performing GridSearchCV for SVM...
Saved probabilities for SVM to /content/SVM_probabilities.csv
Performing GridSearchCV for Decision Tree...
Saved probabilities for Decision Tree to /content/Decision Tree_probabilities.csv
Performing GridSearchCV for Random Forest...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
135 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils

Saved probabilities for Random Forest to /content/Random Forest_probabilities.csv
Performing GridSearchCV for Logistic Regression...
Saved probabilities for Logistic Regression to /content/Logistic Regression_probabilities.csv
Performing GridSearchCV for k-NN...
Saved probabilities for k-NN to /content/k-NN_probabilities.csv
Performing GridSearchCV for Naive Bayes...
Saved probabilities for Naive Bayes to /content/Naive Bayes_probabilities.csv
Performing GridSearchCV for Gradient Boosting...
Saved probabilities for Gradient Boosting to /content/Gradient Boosting_probabilities.csv
Performing GridSearchCV for XGBoost...
Saved probabilities for XGBoost to /content/XGBoost_probabilities.csv
Performing GridSearchCV for LightGBM...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000249 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 598
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Saved probabilities for LightGBM to /content/LightGBM_probabilities.csv
Performing GridSearchCV for CatBoost...
Saved probabilities for CatBoost to /content/CatBoost_probabilities.csv
Performing GridSearchCV for AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Saved probabilities for AdaBoost to /content/AdaBoost_probabilities.csv
Performing GridSearchCV for Neural Network...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Saved probabilities for Neural Network to /content/Neural Network_probabilities.csv
Saved combined dataset to /content/combined_probabilities_GridSearchCV.csv


proposed model for GDC

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                            matthews_corrcoef, cohen_kappa_score, roc_auc_score)

# Load all datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Add target labels (1 for positive, 0 for negative)
main_p['Target'] = 1
main_n['Target'] = 0
validation_p['Target'] = 1
validation_n['Target'] = 0

# Combine main datasets for training and validation datasets for testing
train_data = pd.concat([main_p, main_n], axis=0)
validation_data = pd.concat([validation_p, validation_n], axis=0)

# Shuffle the datasets
train_data = train_data.sample(frac=1, random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Check for and drop any non-numeric columns (except Target)
non_numeric_cols = train_data.select_dtypes(exclude=[np.number]).columns.tolist()
if 'Target' in non_numeric_cols:
    non_numeric_cols.remove('Target')
if non_numeric_cols:
    print(f"Dropping non-numeric columns: {non_numeric_cols}")
    train_data = train_data.drop(columns=non_numeric_cols)
    validation_data = validation_data.drop(columns=non_numeric_cols)

# Separate features and labels
X_train = train_data.drop(columns=['Target']).values
y_train = train_data['Target'].values
X_val = validation_data.drop(columns=['Target']).values
y_val = validation_data['Target'].values

# Check feature dimensions
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

if X_train.shape[1] == 0:
    raise ValueError("No features found! Check your input data.")

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# For GDC features, we'll use a Dense network instead of Conv1D
model = Sequential()

# Input layer
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Hidden layers
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()

# Add early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping]
)

# Evaluate the model on the validation data
val_probabilities = model.predict(X_val).flatten()
val_predictions = (val_probabilities > 0.5).astype(int)

# Calculate all metrics
accuracy = accuracy_score(y_val, val_predictions)
tn, fp, fn, tp = confusion_matrix(y_val, val_predictions).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
mcc = matthews_corrcoef(y_val, val_predictions)
kappa = cohen_kappa_score(y_val, val_predictions)
auc = roc_auc_score(y_val, val_probabilities)

# Print all metrics
print("\nValidation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
print(f"Area Under Curve (AUC): {auc:.4f}")

# Classification report
print("\nClassification Report:\n", classification_report(y_val, val_predictions))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, val_predictions))

Training data shape: (1164, 9)
Validation data shape: (300, 9)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 256)            │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,569 (178.00 KB)

 Trainable params: 44,673 (174.50 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.6165 - loss: 0.7807 - val_accuracy: 0.6433 - val_loss: 0.6308
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6920 - loss: 0.6160 - val_accuracy: 0.7167 - val_loss: 0.5996
Epoch 3/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7240 - loss: 0.5420 - val_accuracy: 0.7333 - val_loss: 0.5828
Epoch 4/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7045 - loss: 0.5402 - val_accuracy: 0.7433 - val_loss: 0.5539
Epoch 5/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7342 - loss: 0.5540 - val_accuracy: 0.7533 - val_loss: 0.5303
Epoch 6/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7450 - loss: 0.5177 - val_accuracy: 0.7400 - val_loss: 0.5180
Epoch 7/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7778 - loss: 0.4730 - val_accuracy: 0.7600 - val_loss: 0.5009
Epoch 8/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7909 - loss: 0.4658 - val_accuracy: 0.7633 - val_lo

In [2]:
#cross validation 5 fold

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (accuracy_score, classification_report,
                           confusion_matrix, matthews_corrcoef,
                           cohen_kappa_score, roc_auc_score)

# Load all datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Add target labels (1 for positive, 0 for negative)
main_p['Target'] = 1
main_n['Target'] = 0
validation_p['Target'] = 1
validation_n['Target'] = 0

# Combine all data for cross-validation
all_data = pd.concat([main_p, main_n, validation_p, validation_n])
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Check and drop non-numeric columns
non_numeric_cols = all_data.select_dtypes(exclude=[np.number]).columns.tolist()
if 'Target' in non_numeric_cols:
    non_numeric_cols.remove('Target')
if non_numeric_cols:
    print(f"Dropping non-numeric columns: {non_numeric_cols}")
    all_data = all_data.drop(columns=non_numeric_cols)

# Separate features and labels
X = all_data.drop(columns=['Target']).values
y = all_data['Target'].values

# Initialize 5-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
results = []

def build_model(input_dim):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_dim,), kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.4),

        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),

        Dense(1, activation='sigmoid')
    ])

    optimizer = Adam(learning_rate=0.0005)
    model.compile(optimizer=optimizer,
                 loss='binary_crossentropy',
                 metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

for train_idx, val_idx in kfold.split(X, y):
    print(f'\n{"="*40}')
    print(f'Training fold {fold_no}')
    print(f'{"="*40}')

    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    # Build model
    model = build_model(X_train.shape[1])

    # Callbacks
    early_stop = EarlyStopping(monitor='val_auc', patience=15, mode='max', restore_best_weights=True)
    checkpoint = ModelCheckpoint(f'best_model_fold{fold_no}.h5', monitor='val_auc', save_best_only=True, mode='max')

    # Class weighting for imbalanced data
    class_counts = np.bincount(y_train)
    class_weight = {0: 1/class_counts[0], 1: 1/class_counts[1]}

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=32,
        callbacks=[early_stop, checkpoint],
        class_weight=class_weight,
        verbose=1
    )

    # Load best model
    model.load_weights(f'best_model_fold{fold_no}.h5')

    # Evaluate
    val_probabilities = model.predict(X_val).flatten()
    val_predictions = (val_probabilities > 0.5).astype(int)

    # Calculate metrics
    tn, fp, fn, tp = confusion_matrix(y_val, val_predictions).ravel()
    metrics = {
        'fold': fold_no,
        'accuracy': accuracy_score(y_val, val_predictions),
        'sensitivity': tp / (tp + fn) if (tp + fn) > 0 else 0,
        'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
        'mcc': matthews_corrcoef(y_val, val_predictions),
        'kappa': cohen_kappa_score(y_val, val_predictions),
        'auc': roc_auc_score(y_val, val_probabilities)
    }

    results.append(metrics)

    print(f'\nFold {fold_no} Results:')
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    print(f"MCC: {metrics['mcc']:.4f}")
    print(f"Sensitivity: {metrics['sensitivity']:.4f}")
    print(f"Specificity: {metrics['specificity']:.4f}")

    fold_no += 1

# Calculate average metrics
print('\n' + '='*50)
print('Cross-Validation Summary')
print('='*50)

avg_metrics = {
    'accuracy': np.mean([r['accuracy'] for r in results]),
    'sensitivity': np.mean([r['sensitivity'] for r in results]),
    'specificity': np.mean([r['specificity'] for r in results]),
    'mcc': np.mean([r['mcc'] for r in results]),
    'kappa': np.mean([r['kappa'] for r in results]),
    'auc': np.mean([r['auc'] for r in results])
}

print(f"\nAverage Metrics Across All Folds:")
print(f"Accuracy: {avg_metrics['accuracy']:.4f}")
print(f"Sensitivity: {avg_metrics['sensitivity']:.4f}")
print(f"Specificity: {avg_metrics['specificity']:.4f}")
print(f"MCC: {avg_metrics['mcc']:.4f}")
print(f"Cohen's Kappa: {avg_metrics['kappa']:.4f}")
print(f"AUC: {avg_metrics['auc']:.4f}")

# Train final model on all data
print("\nTraining final model on all data...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

final_model = build_model(X_scaled.shape[1])

# Final class weights
class_counts = np.bincount(y)
final_class_weight = {0: 1/class_counts[0], 1: 1/class_counts[1]}

history = final_model.fit(
    X_scaled, y,
    epochs=20,
    batch_size=32,
    class_weight=final_class_weight,
    verbose=1
)

# Save final model
final_model.save('gdc_final_model.h5')
print("Final model training complete and saved!")

# Evaluate final model
final_probabilities = final_model.predict(X_scaled).flatten()
final_predictions = (final_probabilities > 0.5).astype(int)

# Calculate final metrics
tn, fp, fn, tp = confusion_matrix(y, final_predictions).ravel()
final_metrics = {
    'accuracy': accuracy_score(y, final_predictions),
    'sensitivity': tp / (tp + fn) if (tp + fn) > 0 else 0,
    'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
    'mcc': matthews_corrcoef(y, final_predictions),
    'kappa': cohen_kappa_score(y, final_predictions),
    'auc': roc_auc_score(y, final_probabilities)
}

print("\nFinal Model Metrics:")
print(f"Accuracy: {final_metrics['accuracy']:.4f}")
print(f"AUC: {final_metrics['auc']:.4f}")
print(f"MCC: {final_metrics['mcc']:.4f}")
print(f"Sensitivity: {final_metrics['sensitivity']:.4f}")
print(f"Specificity: {final_metrics['specificity']:.4f}")


Training fold 1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
34/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5534 - auc: 0.5730 - loss: 2.5179

37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.5535 - auc: 0.5736 - loss: 2.4926 - val_accuracy: 0.5154 - val_auc: 0.6323 - val_loss: 2.5022
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5470 - auc: 0.5998 - loss: 1.6401 - val_accuracy: 0.5017 - val_auc: 0.6194 - val_loss: 1.8794
Epoch 3/20
27/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5726 - auc: 0.6121 - loss: 1.0965

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5719 - auc: 0.6119 - loss: 1.0640 - val_accuracy: 0.5017 - val_auc: 0.6765 - val_loss: 1.4652
Epoch 4/20
35/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5837 - auc: 0.6064 - loss: 0.6879

...

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6656 - auc: 0.7504 - loss: 0.0092 - val_accuracy: 0.5000 - val_auc: 0.7914 - val_loss: 0.7145
Epoch 14/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7213 - auc: 0.7817 - loss: 0.0058 - val_accuracy: 0.5000 - val_auc: 0.7627 - val_loss: 0.7036
Epoch 15/20
36/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6899 - auc: 0.7717 - loss: 0.0038

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6902 - auc: 0.7721 - loss: 0.0038 - val_accuracy: 0.5000 - val_auc: 0.7953 - val_loss: 0.6977
Epoch 16/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6981 - auc: 0.7708 - loss: 0.0026 - val_accuracy: 0.5000 - val_auc: 0.7711 - val_loss: 0.6830
Epoch 17/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7095 - auc: 0.7917 - loss: 0.0019 - val_accuracy: 0.5445 - val_auc: 0.7535 - val_loss: 0.6630
Epoch 18/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7143 - auc: 0.7876 - loss: 0.0015 - val_accuracy: 0.6644 - val_auc: 0.7661 - val_loss: 0.6576
Epoch 19/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7011 - auc: 0.7665 - loss: 0.0013 - val_accuracy: 0.7021 - val_auc: 0.7826 - val_loss: 0.6271
Epoch 20/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6978 - auc: 0.7562 - loss: 0.0012 - val_accuracy: 0.7089 - val_auc: 0.7846 - val_loss: 0.6113
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Fold 5 Results:

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4913 - auc: 0.4969 - loss: 2.4218
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5255 - auc: 0.5055 - loss: 1.4304
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5420 - auc: 0.5711 - loss: 0.8279
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5721 - auc: 0.6144 - loss: 0.4706
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6173 - auc: 0.6567 - loss: 0.2626
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6492 - auc: 0.7073 - loss: 0.1442
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6899 - auc: 0.7433 - loss: 0.0782
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6713 - auc: 0.7448 - loss: 0.0422
Epoch 9/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6823 - auc: 0.7797 - loss: 0.0227
Epoch 10/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6769 - auc: 0.7543 - loss: 0.0124
Epoch 11/20
46/46 ━━━━━━━━━━━━

Final model training complete and saved!
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

Final Model Metrics:
Accuracy: 0.7322
AUC: 0.7811
MCC: 0.4657
Sensitivity: 0.6967
Specificity: 0.7678
